<a href="https://colab.research.google.com/github/RutherfordOtu12/Application-of-GeoAI-A-Comparative-Methodical-urban-mapping-/blob/main/Validation_points_stratified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================
print("=" * 80)
print("STEP 1: LOADING DATA")
print("=" * 80)

# Load the Friedrichshain-Kreuzberg land use data
print("\nLoading FK land use data...")
fk_landuse = gpd.read_file('/content/drive/MyDrive/FK_landuse_clipped.gpkg')  # Adjust path as needed
print(f"✓ Loaded {len(fk_landuse)} land use polygons")
print(f"✓ CRS: {fk_landuse.crs}")
print(f"✓ Columns: {list(fk_landuse.columns)}")

# ============================================================================
# STEP 2: DEFINE CLASSIFICATION MAPPING
# ============================================================================
print("\n" + "=" * 80)
print("STEP 2: DEFINING CLASSIFICATION MAPPING")
print("=" * 80)

# Mapping from official Berlin codes to research classes
official_to_research = {
    10: 1,      # Wohnnutzung → Residential
    21: 2,      # Mischnutzung → Mixed-Use
    30: 3,      # Kerngebietsnutzung → Commercial and Industrial/ Large-Scale Retail/Logistics
    40: 3,      # Gewerbe- und Industrienutzung → Commercial and Industrial/ Large-Scale Retail/Logistics
    50: 4,      # Gemeinbedarfs- und Sondernutzung → Public/Institutional
    60: 3,      # Ver- und Entsorgung → Commercial and Industrial/ Large-Scale Retail/Logistics
    70: 5,      # Wochenendhaus → Green Space and Recreational
    80: 6,      # Verkehrsfläche → Transportation and infrastructure
    100: 5,     # Wald → Green Space and Recreational
    110: None,  # Gewässer → EXCLUDED
    121: 5,     # Grünland → Green Space and Recreational
    122: 5,     # Ackerland → Green Space and Recreational
    130: 5,     # Park/Grünfläche → Green Space and Recreational
    140: None,  # Stadtplatz/Promenade → NEEDS INSPECTION (set to None for now)
    150: 5,     # Friedhof → Green Space and Recreational
    160: 5,     # Kleingartenanlage → Green Space and Recreational
    171: None,  # Brachfläche, vegetationsfrei → EXCLUDED
    172: None,  # Brachfläche, wiesenartiger → EXCLUDED
    173: None,  # Brachfläche, Mischbestand → EXCLUDED
    200: 5,     # Baumschule/Gartenbau → Green Space and Recreational
    90: None,   # Baustelle → EXCLUDED
}

# Class names for reference
research_classes = {
    1: "Residential",
    2: "Mixed-Use",
    3: "Commercial and Industrial/ Large-Scale Retail/Logistics",
    4: "Public/Institutional",
    5: "Green Space and Recreational",
    6: "Transportation and infrastructure",
}

print("\nClassification Mapping:")
print("-" * 80)
for official_code, research_class in sorted(official_to_research.items()):
    if research_class is None:
        status = "EXCLUDED"
    else:
        status = f"→ Class {research_class}: {research_classes[research_class]}"
    print(f"  Code {official_code:3d}: {status}")

# ============================================================================
# STEP 3: PREPARE DATA FOR SAMPLING
# ============================================================================
print("\n" + "=" * 80)
print("STEP 3: PREPARING DATA FOR SAMPLING")
print("=" * 80)

# Identify the column containing official codes
# The column 'schluessel' was identified as the problem in previous run.
# Based on the context of land use, 'nutz' is a more likely candidate for official codes.
# Explicitly set code_column to 'nutz'
code_column = 'nutz'

if code_column not in fk_landuse.columns:
    print(f"\nERROR: The expected code column '{code_column}' was not found!")
    print(f"Available columns: {list(fk_landuse.columns)}")
    exit(1)

print(f"\n✓ Using column '{code_column}' for official codes")

# Map official codes to research classes
print("\nMapping official codes to research classes...")
fk_landuse['official_code'] = fk_landuse[code_column].astype(int)

# --- DEBUGGING: Inspect unique values and mapping results ---
print(f"\nUnique values in '{code_column}' column before mapping:")
print(fk_landuse[code_column].value_counts().sort_index())
# -----------------------------------------------------------

fk_landuse['research_class'] = fk_landuse['official_code'].map(official_to_research)

# --- DEBUGGING: Inspect unique values and mapping results ---
mapped_values_count = fk_landuse['research_class'].value_counts(dropna=False)
print("\nCounts of research_class values after mapping (including NaNs):")
print(mapped_values_count)

non_mapped_codes = fk_landuse[fk_landuse['research_class'].isna()]['official_code'].unique()
print(f"\nOfficial codes that resulted in NaN/None research_class: {sorted(non_mapped_codes.tolist())}")
# -----------------------------------------------------------

# Remove excluded classes
fk_landuse_valid = fk_landuse[fk_landuse['research_class'].notna()].copy()
print(f"✓ Total polygons: {len(fk_landuse)}")
print(f"✓ Valid polygons (after excluding): {len(fk_landuse_valid)}")
print(f"✓ Excluded polygons: {len(fk_landuse) - len(fk_landuse_valid)}")

# Calculate polygon areas for weighted sampling
fk_landuse_valid['area'] = fk_landuse_valid.geometry.area
print(f"✓ Total area: {fk_landuse_valid['area'].sum() / 1e6:.2f} km²")

# ============================================================================
# STEP 4: CALCULATE EXPECTED CLASS DISTRIBUTION
# ============================================================================
print("\n" + "=" * 80)
print("STEP 4: CALCULATING EXPECTED CLASS DISTRIBUTION")
print("=" * 80)

# Group by research class and calculate statistics
class_stats = fk_landuse_valid.groupby('research_class').agg({
    'area': ['sum', 'mean', 'count'],
    'geometry': 'count'
}).round(2)

print("\nClass Statistics (by area):")
print("-" * 80)
total_area = fk_landuse_valid['area'].sum()
for research_class in sorted(fk_landuse_valid['research_class'].unique()):
    class_data = fk_landuse_valid[fk_landuse_valid['research_class'] == research_class]
    class_area = class_data['area'].sum()
    class_pct = (class_area / total_area) * 100
    polygon_count = len(class_data)
    print(f"  Class {research_class}: {research_classes[research_class]}")
    print(f"    - Area: {class_area / 1e6:.2f} km² ({class_pct:.1f}%)")
    print(f"    - Polygons: {polygon_count}")

# ============================================================================
# STEP 5: STRATIFIED RANDOM POINT GENERATION
# ============================================================================
print("\n" + "=" * 80)
print("STEP 5: GENERATING STRATIFIED RANDOM VALIDATION POINTS")
print("=" * 80)

# Target number of validation points
target_points = 300

# Expected distribution (from methodology)
expected_distribution = {
    1: 0.45,  # Residential: 40-50%
    2: 0.10,  # Mixed-Use: 8-12%
    3: 0.20,  # Commercial and Industrial/ Large-Scale Retail/Logistics: 15-25%
    4: 0.075, # Public/Institutional: 5-10%
    5: 0.125, # Green Space and Recreational: 10-15%
    6: 0.075, # Transportation and infrastructure: 5-10%
}

# Calculate target points per class
target_per_class = {}
for research_class, pct in expected_distribution.items():
    target_per_class[research_class] = int(target_points * pct)

print(f"\nTarget total points: {target_points}")
print("\nTarget distribution by class:")
print("-" * 80)
total_target = 0
for research_class in sorted(target_per_class.keys()):
    target = target_per_class[research_class]
    pct = (target / target_points) * 100
    print(f"  Class {research_class}: {research_classes[research_class]}")
    print(f"    - Target points: {target} ({pct:.1f}%)")
    total_target += target

print(f"\nTotal target: {total_target}")

# Generate random points within each class
validation_points = []
validation_metadata = []

print("\nGenerating random points by class...")
print("-" * 80)

for research_class in sorted(fk_landuse_valid['research_class'].unique()):
    class_data = fk_landuse_valid[fk_landuse_valid['research_class'] == research_class]
    target = target_per_class.get(research_class, 0)

    print(f"\n  Class {research_class}: {research_classes[research_class]}")
    print(f"    - Generating {target} points from {len(class_data)} polygons...")

    # Generate random points
    class_points = []
    for idx, row in class_data.iterrows():
        # Calculate expected number of points in this polygon
        polygon_area = row['area']
        class_area = class_data['area'].sum()
        points_in_polygon = int(target * (polygon_area / class_area))

        # Generate random points within polygon
        for _ in range(points_in_polygon):
            # Generate random point within polygon bounds
            minx, miny, maxx, maxy = row['geometry'].bounds
            while True:
                x = np.random.uniform(minx, maxx)
                y = np.random.uniform(miny, maxy)
                point = Point(x, y)
                if row['geometry'].contains(point):
                    class_points.append(point)
                    break

    # If we need more points, add random ones
    if len(class_points) < target:
        for _ in range(target - len(class_points)):
            random_polygon = class_data.sample(1).iloc[0]
            minx, miny, maxx, maxy = random_polygon['geometry'].bounds
            while True:
                x = np.random.uniform(minx, maxx)
                y = np.random.uniform(miny, maxy)
                point = Point(x, y)
                if random_polygon['geometry'].contains(point):
                    class_points.append(point)
                    break

    # Trim to target if too many
    class_points = class_points[:target]

    print(f"    - Generated: {len(class_points)} points")

    # Add to validation set
    for i, point in enumerate(class_points):
        validation_points.append(point)
        validation_metadata.append({
            'point_id': len(validation_points),
            'research_class': research_class,
            'class_name': research_classes[research_class],
            'official_code': None,  # Will be assigned via spatial join
        })

print(f"\n✓ Total validation points generated: {len(validation_points)}")

# ============================================================================
# STEP 6: CREATE GEODATAFRAME AND ASSIGN OFFICIAL CODES
# ============================================================================
print("\n" + "=" * 80)
print("STEP 6: CREATING GEODATAFRAME AND ASSIGNING OFFICIAL CODES")
print("=" * 80)

# Create GeoDataFrame
validation_gdf = gpd.GeoDataFrame(
    validation_metadata,
    geometry=validation_points,
    crs=fk_landuse.crs
)

print(f"\n✓ Created GeoDataFrame with {len(validation_gdf)} points")
print(f"✓ CRS: {validation_gdf.crs}")

# Spatial join to get official codes
print("\nPerforming spatial join to assign official codes...")
validation_gdf = gpd.sjoin(
    validation_gdf,
    fk_landuse[['official_code', 'geometry']],
    how='left',
    predicate='within'
)

# Rename the joined column
validation_gdf = validation_gdf.rename(columns={'official_code_right': 'official_code_assigned'})
validation_gdf = validation_gdf.drop(columns=['index_right'], errors='ignore')

print("✓ Spatial join complete")

# ============================================================================
# STEP 7: QUALITY ASSURANCE
# ============================================================================
print("\n" + "=" * 80)
print("STEP 7: QUALITY ASSURANCE")
print("=" * 80)

print("\nValidation Point Summary:")
print("-" * 80)
for research_class in sorted(validation_gdf['research_class'].unique()):
    class_points = validation_gdf[validation_gdf['research_class'] == research_class]
    print(f"  Class {research_class}: {research_classes[research_class]}")
    print(f"    - Points: {len(class_points)}")
    print(f"    - Official codes assigned: {class_points['official_code_assigned'].notna().sum()}")

print(f"\n✓ Total validation points: {len(validation_gdf)}")
print(f"✓ Points with official codes: {validation_gdf['official_code_assigned'].notna().sum()}")
print(f"✓ Points without codes: {validation_gdf['official_code_assigned'].isna().sum()}")

# ============================================================================
# STEP 8: SAVE OUTPUTS
# ============================================================================
print("\n" + "=" * 80)
print("STEP 8: SAVING OUTPUTS")
print("=" * 80)

# Save as GeoPackage
output_gpkg = 'FK_validation_points_300.gpkg'
validation_gdf.to_file(output_gpkg, layer='validation_points')
print(f"\n✓ Saved to: {output_gpkg}")

# Save as CSV for spreadsheet use
output_csv = 'FK_validation_points_300.csv'
validation_csv = validation_gdf.copy()
validation_csv['geometry'] = validation_csv['geometry'].astype(str)
validation_csv.to_csv(output_csv, index=False)
print(f"✓ Saved to: {output_csv}")

# Save as Shapefile
output_shp = 'FK_validation_points_300.shp'
validation_gdf.to_file(output_shp)
print(f"✓ Saved to: {output_shp}")

# Create summary report
print("\n" + "=" * 80)
print("STEP 9: GENERATING SUMMARY REPORT")
print("=" * 80)

summary_report = f"""
VALIDATION DATASET GENERATION REPORT
=====================================
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

DATASET SUMMARY
---------------
Total Validation Points: {len(validation_gdf)}
CRS: {validation_gdf.crs}
Study Area: Friedrichshain-Kreuzberg, Berlin

CLASS DISTRIBUTION
------------------
"""

for research_class in sorted(validation_gdf['research_class'].unique()):
    class_points = validation_gdf[validation_gdf['research_class'] == research_class]
    pct = (len(class_points) / len(validation_gdf)) * 100
    summary_report += f"\nClass {research_class}: {research_classes[research_class]}\n"
    summary_report += f"  Points: {len(class_points)} ({pct:.1f}%)\n"
    summary_report += f"  Official codes assigned: {class_points['official_code_assigned'].notna().sum()}\n"

summary_report += f"""

OUTPUT FILES
------------
1. {output_gpkg} - GeoPackage format (recommended for GIS)
2. {output_shp} - Shapefile format
3. {output_csv} - CSV format (for spreadsheets)
4. FK_validation_summary.txt - This report

NEXT STEPS
----------
1. Review the validation points in QGIS
2. Verify spatial distribution across study area
3. Check for any points outside study area
4. Prepare field collection materials
5. Begin ground truth verification

NOTES
-----
- Points were generated using stratified random sampling
- Expected distribution was based on land use class proportions
- Official codes assigned via spatial join with land use polygons
- Some points may fall on boundaries; verify in field
"""

report_file = 'FK_validation_summary.txt'
with open(report_file, 'w') as f:
    f.write(summary_report)

print(f"✓ Saved summary report to: {report_file}")

print("\n" + "=" * 80)
print("VALIDATION POINT GENERATION COMPLETE!")
print("=" * 80)
print(f"\nGenerated {len(validation_gdf)} validation points")
print(f"Files saved:")
print(f"  - {output_gpkg}")
print(f"  - {output_shp}")
print(f"  - {output_csv}")
print(f"  - {report_file}")

STEP 1: LOADING DATA

Loading FK land use data...
✓ Loaded 851 land use polygons
✓ CRS: EPSG:25833
✓ Columns: ['schluessel', 'flalle', 'bez', 'bezirk', 'woz', 'woz_name', 'ewoz_name', 'grz', 'grz_name', 'egrz_name', 'typ', 'typklar', 'etypklar', 'nutz', 'nutzung', 'enutzung', 'nutz_bauvor', 'nutzung_bauvor', 'enutzung_bauvor', 'ststrnr', 'ststrname', 'eststrname', 'vg_2021', 'probau_2021', 'provgneu_2021', 'vg_0_2021', 'provgneu_0_2021', 'kl1', 'kl2', 'kl3', 'kl4', 'geometry']

STEP 2: DEFINING CLASSIFICATION MAPPING

Classification Mapping:
--------------------------------------------------------------------------------
  Code  10: → Class 1: Residential
  Code  21: → Class 2: Mixed-Use
  Code  30: → Class 3: Commercial and Industrial/ Large-Scale Retail/Logistics
  Code  40: → Class 3: Commercial and Industrial/ Large-Scale Retail/Logistics
  Code  50: → Class 4: Public/Institutional
  Code  60: → Class 3: Commercial and Industrial/ Large-Scale Retail/Logistics
  Code  70: → Class 5: